# Backtest Trading Report

This notebook analyzes the backtest aligned to your trading flow:
- Predict after market close on day D
- Buy at next trading day open (D+1)
- Sell at next trading day close (D+1)
- Capital locked for T+3 trading days after sell (weekends skipped)

Assumptions:
- Equal weight across 10 picks
- No transaction costs/slippage


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

picks_path = Path(r'..\models\saved\backtest_20260120_142153_picks.csv')
daily_path = Path(r'..\models\saved\backtest_20260120_142153.csv')

picks = pd.read_csv(picks_path, parse_dates=['date', 'target_date'])
daily = pd.read_csv(daily_path, parse_dates=['test_date', 'target_date'])

picks['actual_return'] = pd.to_numeric(picks['actual_return'], errors='coerce')
daily['precision_at_k'] = pd.to_numeric(daily['precision_at_k'], errors='coerce')

picks.head()

In [ ]:
# Summary metrics
total_trade_days = picks['date'].nunique()
total_picks = len(picks)
unique_symbols = picks['symbol'].nunique()

daily_returns = picks.groupby('date')['actual_return'].mean().dropna()
cumulative_return = (1 + daily_returns).prod() - 1 if not daily_returns.empty else np.nan

pick_win_rate = (picks['actual_return'] > 0).mean()
hit_rate_days = picks.groupby('date')['is_correct'].any().mean()

summary = pd.Series({
    'total_trade_days': total_trade_days,
    'total_picks': total_picks,
    'unique_symbols': unique_symbols,
    'daily_avg_return': daily_returns.mean(),
    'daily_median_return': daily_returns.median(),
    'cumulative_return_compounded': cumulative_return,
    'pick_win_rate': pick_win_rate,
    'hit_rate_days': hit_rate_days,
})
summary

In [ ]:
# Cumulative return curve (equal-weight picks)
cum_curve = (1 + daily_returns).cumprod()
plt.figure(figsize=(10, 4))
plt.plot(cum_curve.index, cum_curve.values, marker='o')
plt.title('Cumulative Return (Compounded)')
plt.ylabel('Growth of 1.0x')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Daily precision@10
plt.figure(figsize=(10, 4))
plt.plot(daily['test_date'], daily['precision_at_k'], marker='o')
plt.title('Daily Precision@10')
plt.ylabel('Precision@10')
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Histogram of pick returns
plt.figure(figsize=(8, 4))
picks['actual_return'].dropna().hist(bins=30)
plt.title('Distribution of Pick Returns (Open->Close)')
plt.xlabel('Return')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Top symbols by average return
symbol_stats = picks.groupby('symbol').agg(
    picks=('symbol', 'count'),
    avg_return=('actual_return', 'mean'),
    median_return=('actual_return', 'median'),
    win_rate=('actual_return', lambda s: (s > 0).mean()),
)

top_by_return = symbol_stats.sort_values('avg_return', ascending=False).head(15)
top_by_return

In [ ]:
# Bar chart: top symbols by average return
plt.figure(figsize=(10, 5))
plt.bar(top_by_return.index, top_by_return['avg_return'])
plt.title('Top 15 Symbols by Average Return')
plt.ylabel('Average Return')
plt.xticks(rotation=45, ha='right')
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Top symbols by win rate (min 3 picks)
top_by_winrate = symbol_stats[symbol_stats['picks'] >= 3].sort_values(
    ['win_rate', 'avg_return'],
    ascending=False
).head(15)
top_by_winrate